In [ ]:
import sys
import os
import pandas as pd
import json
from dotenv import load_dotenv

import logging
logging.getLogger("asyncio").setLevel(logging.CRITICAL)

load_dotenv()

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

from core.data_sources import CLOBDataSource
from core.backtesting import BacktestingEngine
from core.services.backend_api_client import BackendAPIClient

backend_api = BackendAPIClient(host=os.getenv("TRADING_HOST", "localhost"))
clob = CLOBDataSource()
backtesting = BacktestingEngine(root_path=root_path, load_cached_data=True)

await backend_api.list_databases()

In [ ]:
selected_dbs = ['bots/archived/hummingbot-test_1_orugol-2024.11.27_01.45/data/test_1_orugol-2024.sqlite']
fetch_trades = False


# Get selected dbs
dbs = await backend_api.read_databases(selected_dbs)

# Parse internal fields
selected_db = dbs[0].copy()
for table_name in selected_db["tables"].keys():
    selected_db["tables"][table_name] = pd.DataFrame(json.loads(selected_db["tables"][table_name]))

# Load into dfs executors and controllers
executors = selected_db["tables"]["executors"]
controllers = selected_db["tables"]["controllers"]

# Extract controller ids for analysis
controller_ids = list(controllers["id"].unique())

# Choose a controller to analyse
selected_controller = "8uCGoaQhJHirY895PSqZn1mmXkxxCBQr23RvXHYevNGV"
selected_executors = executors[executors["controller_id"] == selected_controller]
selected_config = json.loads(controllers.loc[controllers["id"] == selected_controller, "config"][0])

# Setup configs
connector_name = selected_config["connector_name"]
trading_pair = selected_config["trading_pair"]
interval = "1s"
start_time = selected_executors.timestamp.min() - 60 * 10
end_time = selected_executors.close_timestamp.max() + 60 * 10
from_trades = True

if fetch_trades:
    candles = await clob.get_candles(connector_name=connector_name,
                                     trading_pair=trading_pair,
                                     interval=interval,
                                     start_time=start_time,
                                     end_time=end_time,
                                     from_trades=from_trades)
    candles["quote_asset_volume"] = candles["close"] * candles["volume"]
    missing_cols = ['n_trades', 'taker_buy_base_volume', 'taker_buy_quote_volume']
    for connector_name, trading_pair, interval in clob._candles_cache:
        clob._candles_cache[(connector_name, trading_pair, interval)] = candles.data
    clob.dump_candles_cache(root_path)
else:
    clob.load_candles_cache(root_path)
    backtesting.load_candles_cache_by_connector_pair(connector_name=connector_name,
                                                     trading_pair=trading_pair,
                                                     root_path=root_path)
    # TODO: remove this fixed values after solving issue with missmatching interval in candles downloading
    candles = clob._candles_cache[(connector_name, trading_pair, interval)].copy()
    fixed_start_time = candles.timestamp.min()
    fixed_end_time = candles.timestamp.max()

    # Get config instance and run bt_results
    config_instance = backtesting.get_controller_config_instance_from_dict(selected_config)
    backtesting_result = await backtesting.run_backtesting(config_instance, candles.timestamp.min(), candles.timestamp.max(), interval)


In [ ]:
backtesting_result.get_backtesting_figure()

In [ ]:
from hummingbot.connector.connector_base import TradeType
from hummingbot.strategy_v2.models.executors_info import ExecutorInfo

live_executors = selected_executors.copy()
live_executors = live_executors[(live_executors["timestamp"] >= fixed_start_time) & (live_executors["timestamp"] <= fixed_end_time)]
live_executors["config"] = live_executors["config"].apply(lambda x: json.loads(x))
live_executors["custom_info"] = live_executors["custom_info"].apply(lambda x: json.loads(x))
for index, executor in live_executors.iterrows():
    if executor["custom_info"].get("close_price") is None:
        net_pnl_pct = executor.net_pnl_pct if executor.custom_info["side"] == TradeType.BUY else - executor.net_pnl_pct
        executor["custom_info"]["close_price"] = (1 + net_pnl_pct) * executor.custom_info["current_position_average_price"]

# TODO: remove after using dbs with close_price in custom_info
live_executors_info_list = [ExecutorInfo(**executor) for executor in live_executors.to_dict(orient="records")]
live_executors_info_list

In [ ]:
bt_fig = backtesting_result.get_backtesting_figure()
backtesting_result._add_executors_trace(fig=bt_fig,
                                        executors=live_executors_info_list,
                                        line_style="filled")